## Imports

In [1]:
from scipy.spatial import cKDTree

from utils import *

## Zone centroids

In [2]:
pts = (gpd.read_parquet('data/zones.parquet', columns=[
    'geoid', 'level', 'city', 'geometry'])
       .set_index(['geoid', 'level', 'city']).to_crs(CRS_M)
       .centroid.get_coordinates().reset_index())
pts = pdf2gdf(pts, 'x', 'y', CRS_M)
latlon = (pts.to_crs(CRS_DEG).get_coordinates()
          .rename(columns=D(x='lon', y='lat')))
pts = (pd.concat([pts.drop(columns='geometry'), latlon], axis=1)
       .astype(D(level=str, city=str, lon=F32, lat=F32))).disp() # 8s

143,971 rows x 7 cols; Memory: 29.9 MiB


,geoid,level,city,x,y,lon,lat
,<object>,<object>,<object>,<float64>,<float64>,<float32>,<float32>
0,42007,County,Pittsburgh,-8944439.202797,4965621.768359,-80.349266,40.682495


## Filter OD pairs
From all the possible OD pairs for each combination of city, mode, and spatial level, filter the ones which are feasibly reachable within a given time threshold (60 min) using the Haversine distance between the centroids of the OD pairs using the maximum modal speed.

In [3]:
max_time = 60 * 60 # maximum allowed travel time (seconds)
max_speeds = D(Bike=16, Drive=70, Transit=30, Walk=3.1) # in mph

In [4]:
# pbar = tqdm(list(it.product(max_speeds, pts.groupby('city'))))
# for mode, (city, df) in pbar:
#     pbar.set_description(f'{city}: {mode}')
#     max_dist = max_speeds[mode] * MPH2MPS * max_time
#     odp = []
#     for level, df2 in df.groupby('level'):
#         df3 = df2.reset_index().rename_axis('id')[['geoid', 'lon', 'lat']]
#         src = df3.rename(columns=lambda x: 'src_' + x)
#         trg = df3.rename(columns=lambda x: 'trg_' + x)
#         tree = cKDTree(df2[['x', 'y']])
#         od = tree.query_ball_point(df2[['x', 'y']], max_dist, workers=8)
#         od = Pdf(od, columns=['trg']).rename_axis('src').reset_index()
#         od = od.explode('trg').query('src != trg').reset_index(drop=1)
#         od = od.merge(src, left_on='src', right_index=True)
#         od = od.merge(trg, left_on='trg', right_index=True)
#         od = od.drop(columns=['src', 'trg'])
#         od = od.rename(columns=D(src_geoid='src', trg_geoid='trg'))
#         od.insert(0, 'level', level)
#         odp.append(od)
#     odp = pd.concat(odp).reset_index(drop=1).astype(D(level=CAT))
#     outfile = city.lower().replace(' ', '-') + f'_{mode.lower()}.parquet'
#     odp.to_parquet(mkdir('data/odp') / outfile, compression='gzip') # 25m19s